# Chapter 48: Building a Social Media Platform

Social media platforms represent the pinnacle of web application complexity: massive scale user-generated content, graph-based relationship models, real-time interaction requirements, and algorithmic feed generation. This chapter explores architectural patterns for building modern social platforms using Next.js, including feed distribution strategies, follower graph management, content moderation pipelines, and notification systems that handle high-write workloads.

By the end of this chapter, you'll master graph database modeling for social connections, implement fan-out strategies for feed generation, build real-time interaction systems for likes and comments, create scalable notification architectures with delivery guarantees, deploy AI-powered content moderation, and optimize for high-concurrency social workloads with cursor-based pagination.

## 48.1 Feed Architecture

Designing data pipelines that balance read performance with write consistency.

### Database Schema for Social Graphs

Relational modeling of follower relationships and content distribution:

```typescript
// prisma/schema.prisma
model User {
  id            String    @id @default(cuid())
  username      String    @unique
  email         String    @unique
  name          String?
  bio           String?   @db.Text
  avatar        String?
  verified      Boolean   @default(false)
  private       Boolean   @default(false)
  
  // Social graph - Following relationships
  following     Follow[]  @relation("UserFollowing")
  followers     Follow[]  @relation("UserFollowers")
  
  // Content
  posts         Post[]
  likes         Like[]
  comments      Comment[]
  bookmarks     Bookmark[]
  
  // Feed caches
  feedItems     FeedItem[]
  
  // Notifications
  notifications Notification[]
  sentNotifications Notification[] @relation("ActorNotifications")
  
  // Settings
  preferences   UserPreferences?
  
  createdAt     DateTime  @default(now())
  updatedAt     DateTime  @updatedAt

  @@index([username])
  @@index([createdAt])
}

model Follow {
  id          String   @id @default(cuid())
  followerId  String
  followingId String
  status      FollowStatus @default(ACCEPTED)
  createdAt   DateTime @default(now())
  
  follower    User     @relation("UserFollowing", fields: [followerId], references: [id], onDelete: Cascade)
  following   User     @relation("UserFollowers", fields: [followingId], references: [id], onDelete: Cascade)

  @@unique([followerId, followingId])
  @@index([followingId, createdAt])
  @@index([followerId, createdAt])
}

enum FollowStatus {
  PENDING   // For private accounts
  ACCEPTED
  BLOCKED
}

model Post {
  id          String   @id @default(cuid())
  authorId    String
  content     String?  @db.Text
  mediaUrls   String[] // Array of S3/Cloudflare R2 URLs
  
  // Engagement metrics (denormalized for performance)
  likesCount    Int    @default(0)
  commentsCount Int    @default(0)
  repostsCount  Int    @default(0)
  viewsCount    Int    @default(0)
  
  // Relationships
  author      User     @relation(fields: [authorId], references: [id], onDelete: Cascade)
  likes       Like[]
  comments    Comment[]
  bookmarks   Bookmark[]
  feedItems   FeedItem[]
  
  // Content moderation
  moderationStatus ModerationStatus @default(PENDING)
  moderationLabels Json?
  
  // Metadata
  location    String?
  replyToId   String?
  replyTo     Post?    @relation("ReplyThread", fields: [replyToId], references: [id])
  replies     Post[]   @relation("ReplyThread")
  
  createdAt   DateTime @default(now())
  updatedAt   DateTime @updatedAt

  @@index([authorId, createdAt])
  @@index([createdAt])
  @@index([moderationStatus, createdAt])
}

model FeedItem {
  id          String   @id @default(cuid())
  userId      String   // Who sees this in their feed
  postId      String   // What they see
  authorId    String   // Who created the content (for quick filtering)
  score       Float    @default(0) // Algorithmic ranking score
  reason      FeedReason @default(FOLLOWING)
  
  user        User     @relation(fields: [userId], references: [id], onDelete: Cascade)
  post        Post     @relation(fields: [postId], references: [id], onDelete: Cascade)
  
  createdAt   DateTime @default(now())
  
  @@unique([userId, postId])
  @@index([userId, score, createdAt]) // For ranked feeds
  @@index([userId, createdAt])        // For chronological feeds
}

enum FeedReason {
  FOLLOWING
  RECOMMENDED
  TRENDING
  REPOSTED
}
```

### Fan-out on Write Strategy

Pre-computing feeds for active users to optimize read performance:

```typescript
// lib/feed/fanout.ts
import { prisma } from '@/shared/lib/db';

type PostWithAuthor = {
  id: string;
  authorId: string;
  content: string | null;
  createdAt: Date;
  author: {
    id: string;
    followers: Array<{ followerId: string }>;
  };
};

export async function fanOutPost(post: PostWithAuthor) {
  // Get all followers
  const followers = await prisma.follow.findMany({
    where: { 
      followingId: post.authorId,
      status: 'ACCEPTED'
    },
    select: { followerId: true },
  });

  // For users with < 10k followers: Fan-out on write
  if (followers.length < 10000) {
    const feedItems = followers.map(follower => ({
      userId: follower.followerId,
      postId: post.id,
      authorId: post.authorId,
      reason: 'FOLLOWING' as const,
      score: calculateFeedScore(post),
    }));

    // Batch insert in chunks of 1000
    const chunkSize = 1000;
    for (let i = 0; i < feedItems.length; i += chunkSize) {
      const chunk = feedItems.slice(i, i + chunkSize);
      await prisma.feedItem.createMany({
        data: chunk,
        skipDuplicates: true,
      });
    }
  } else {
    // For celebrities (>10k followers): Fan-out on read
    // Store in "popular posts" cache for pull-based feed generation
    await redis.zadd(
      'popular_posts',
      calculateFeedScore(post),
      post.id
    );
  }
}

function calculateFeedScore(post: any): number {
  // Algorithm: Recency + Engagement potential
  const hoursSincePost = (Date.now() - new Date(post.createdAt).getTime()) / (1000 * 60 * 60);
  const recencyScore = Math.exp(-hoursSincePost / 24); // Decay over 24 hours
  
  // Author influence (could be cached on user table)
  const authorScore = 1; // Simplified
  
  return recencyScore * authorScore * 1000;
}

// Background job for large fan-outs
// app/api/jobs/fanout/route.ts
import { NextResponse } from 'next/server';
import { Queue } from 'bullmq';
import { redis } from '@/shared/lib/redis';

const fanOutQueue = new Queue('fanout', { connection: redis });

export async function POST(request: Request) {
  const { postId, authorId } = await request.json();
  
  // Add to queue for async processing
  await fanOutQueue.add('fanout-post', {
    postId,
    authorId,
    cursor: 0, // For pagination through followers
  }, {
    attempts: 3,
    backoff: {
      type: 'exponential',
      delay: 1000,
    },
  });

  return NextResponse.json({ queued: true });
}
```

### Hybrid Feed Generation

Combining pre-computed and real-time content:

```typescript
// lib/feed/generator.ts
import { prisma } from '@/shared/lib/db';

interface FeedOptions {
  userId: string;
  cursor?: string;
  limit?: number;
  algorithm?: 'chronological' | 'ranked';
}

export async function getFeed({ userId, cursor, limit = 20, algorithm = 'chronological' }: FeedOptions) {
  // Get feed items from pre-computed table
  const feedItems = await prisma.feedItem.findMany({
    where: { 
      userId,
      ...(cursor && {
        createdAt: { lt: new Date(cursor) }
      })
    },
    include: {
      post: {
        include: {
          author: {
            select: {
              id: true,
              username: true,
              name: true,
              avatar: true,
              verified: true,
            },
          },
          _count: {
            select: {
              likes: true,
              comments: true,
              replies: true,
            },
          },
        },
      },
    },
    orderBy: algorithm === 'ranked' 
      ? [{ score: 'desc' }, { createdAt: 'desc' }]
      : { createdAt: 'desc' },
    take: limit,
  });

  // For high-follower accounts, supplement with pull-based content
  const following = await prisma.follow.findMany({
    where: { followerId: userId },
    select: { followingId: true },
  });

  const followingIds = following.map(f => f.followingId);

  // Get any posts that might have been missed (recent posts since last fan-out)
  const recentPosts = await prisma.post.findMany({
    where: {
      authorId: { in: followingIds },
      createdAt: { gt: new Date(Date.now() - 5 * 60 * 1000) }, // Last 5 minutes
      NOT: {
        id: { in: feedItems.map(fi => fi.postId) }
      }
    },
    include: {
      author: {
        select: {
          id: true,
          username: true,
          name: true,
          avatar: true,
          verified: true,
        },
      },
    },
    orderBy: { createdAt: 'desc' },
    take: 5,
  });

  // Merge and sort
  const merged = [
    ...feedItems.map(fi => fi.post),
    ...recentPosts
  ].sort((a, b) => new Date(b.createdAt).getTime() - new Date(a.createdAt).getTime());

  // Get next cursor
  const nextCursor = merged.length > 0 
    ? merged[merged.length - 1].createdAt.toISOString()
    : null;

  return {
    posts: merged.slice(0, limit),
    nextCursor,
  };
}
```

## 48.2 User Profiles & Social Graph

Managing follower relationships and profile data.

### Follow/Unfollow Actions

```typescript
// features/social/actions.ts
'use server';

import { revalidatePath } from 'next/cache';
import { prisma } from '@/shared/lib/db';
import { getServerSession } from 'next-auth';
import { authOptions } from '@/app/api/auth/[...nextauth]/route';
import { createNotification } from '@/lib/notifications';

export async function toggleFollow(targetUserId: string) {
  const session = await getServerSession(authOptions);
  if (!session?.user) throw new Error('Unauthorized');

  const followerId = session.user.id;
  if (followerId === targetUserId) throw new Error('Cannot follow yourself');

  const existing = await prisma.follow.findUnique({
    where: {
      followerId_followingId: {
        followerId,
        followingId: targetUserId,
      },
    },
  });

  if (existing) {
    // Unfollow
    await prisma.follow.delete({
      where: { id: existing.id },
    });
    
    return { following: false };
  } else {
    // Check if target is private
    const targetUser = await prisma.user.findUnique({
      where: { id: targetUserId },
      select: { private: true, id: true },
    });

    const isPrivate = targetUser?.private ?? false;

    // Create follow request
    await prisma.follow.create({
      data: {
        followerId,
        followingId: targetUserId,
        status: isPrivate ? 'PENDING' : 'ACCEPTED',
      },
    });

    // Send notification
    if (!isPrivate) {
      await createNotification({
        recipientId: targetUserId,
        actorId: followerId,
        type: 'FOLLOW',
        entityType: 'user',
        entityId: followerId,
      });
    }

    revalidatePath(`/users/${targetUserId}`);
    return { following: true, pending: isPrivate };
  }
}

export async function getFollowers(userId: string, cursor?: string, limit: number = 20) {
  const followers = await prisma.follow.findMany({
    where: { 
      followingId: userId,
      status: 'ACCEPTED',
      ...(cursor && {
        createdAt: { lt: new Date(cursor) }
      })
    },
    include: {
      follower: {
        select: {
          id: true,
          username: true,
          name: true,
          avatar: true,
          bio: true,
        },
      },
    },
    orderBy: { createdAt: 'desc' },
    take: limit,
  });

  return {
    users: followers.map(f => f.follower),
    nextCursor: followers.length > 0 
      ? followers[followers.length - 1].createdAt.toISOString()
      : null,
  };
}

export async function getFollowingStatus(currentUserId: string | undefined, targetUserId: string) {
  if (!currentUserId) return { isFollowing: false, isFollowedBy: false };

  const [following, followedBy] = await Promise.all([
    prisma.follow.findUnique({
      where: {
        followerId_followingId: {
          followerId: currentUserId,
          followingId: targetUserId,
        },
      },
    }),
    prisma.follow.findUnique({
      where: {
        followerId_followingId: {
          followerId: targetUserId,
          followingId: currentUserId,
        },
      },
    }),
  ]);

  return {
    isFollowing: following?.status === 'ACCEPTED',
    isPending: following?.status === 'PENDING',
    isFollowedBy: !!followedBy,
  };
}
```

### Profile Page with Infinite Scroll

```typescript
// app/[username]/page.tsx
import { notFound } from 'next/navigation';
import { prisma } from '@/shared/lib/db';
import { getServerSession } from 'next-auth';
import { authOptions } from '@/app/api/auth/[...nextauth]/route';
import { ProfileHeader } from '@/features/profile/profile-header';
import { PostGrid } from '@/features/posts/post-grid';
import { getFollowingStatus } from '@/features/social/actions';

interface ProfilePageProps {
  params: { username: string };
}

export default async function ProfilePage({ params }: ProfilePageProps) {
  const session = await getServerSession(authOptions);
  
  const user = await prisma.user.findUnique({
    where: { username: params.username },
    include: {
      _count: {
        select: {
          posts: true,
          followers: { where: { status: 'ACCEPTED' } },
          following: { where: { status: 'ACCEPTED' } },
        },
      },
    },
  });

  if (!user) notFound();

  // Check if private and not followed
  const relationship = await getFollowingStatus(session?.user?.id, user.id);
  
  if (user.private && !relationship.isFollowing && user.id !== session?.user?.id) {
    return <PrivateProfile user={user} relationship={relationship} />;
  }

  // Initial posts load
  const posts = await prisma.post.findMany({
    where: { 
      authorId: user.id,
      replyToId: null, // Top-level posts only
      moderationStatus: 'APPROVED',
    },
    orderBy: { createdAt: 'desc' },
    take: 12,
  });

  return (
    <div className="max-w-4xl mx-auto">
      <ProfileHeader 
        user={user} 
        stats={user._count}
        relationship={relationship}
      />
      
      <PostGrid 
        initialPosts={posts}
        userId={user.id}
        currentUserId={session?.user?.id}
      />
    </div>
  );
}
```

## 48.3 Post Management

Content creation with media handling and validation.

### Media Upload Pipeline

```typescript
// features/posts/actions.ts
'use server';

import { put } from '@vercel/blob';
import { prisma } from '@/shared/lib/db';
import { getServerSession } from 'next-auth';
import { authOptions } from '@/app/api/auth/[...nextauth]/route';
import { revalidatePath } from 'next/cache';
import { moderateContent } from '@/lib/moderation';
import { fanOutPost } from '@/lib/feed/fanout';

export async function createPost(formData: FormData) {
  const session = await getServerSession(authOptions);
  if (!session?.user) throw new Error('Unauthorized');

  const content = formData.get('content') as string;
  const files = formData.getAll('media') as File[];
  const replyToId = formData.get('replyToId') as string | null;

  // Validate content
  if (!content && files.length === 0) {
    throw new Error('Post cannot be empty');
  }

  // Upload media files
  const mediaUrls: string[] = [];
  for (const file of files) {
    if (file.size > 10 * 1024 * 1024) { // 10MB limit
      throw new Error(`File ${file.name} exceeds 10MB limit`);
    }

    const blob = await put(file.name, file, {
      access: 'public',
      contentType: file.type,
    });
    
    mediaUrls.push(blob.url);
  }

  // Content moderation check
  const moderation = await moderateContent(content || '');
  
  // Create post
  const post = await prisma.post.create({
    data: {
      authorId: session.user.id,
      content,
      mediaUrls,
      replyToId,
      moderationStatus: moderation.flagged ? 'PENDING_REVIEW' : 'APPROVED',
      moderationLabels: moderation.labels,
    },
    include: {
      author: {
        include: {
          followers: {
            where: { status: 'ACCEPTED' },
            select: { followerId: true },
          },
        },
      },
    },
  });

  // Update author's post count (if caching)
  await prisma.user.update({
    where: { id: session.user.id },
    data: { postsCount: { increment: 1 } },
  });

  // Fan-out to feeds (async)
  if (post.moderationStatus === 'APPROVED') {
    await fanOutPost(post);
  }

  revalidatePath(`/users/${session.user.username}`);
  if (replyToId) {
    revalidatePath(`/posts/${replyToId}`);
  }

  return post;
}

export async function deletePost(postId: string) {
  const session = await getServerSession(authOptions);
  if (!session?.user) throw new Error('Unauthorized');

  const post = await prisma.post.findUnique({
    where: { id: postId },
    select: { authorId: true },
  });

  if (!post || post.authorId !== session.user.id) {
    throw new Error('Not authorized to delete this post');
  }

  // Soft delete (or hard delete with cascade)
  await prisma.post.update({
    where: { id: postId },
    data: { 
      moderationStatus: 'DELETED_BY_USER',
      content: '[deleted]',
      mediaUrls: [],
    },
  });

  // Remove from feeds
  await prisma.feedItem.deleteMany({
    where: { postId },
  });

  revalidatePath(`/users/${session.user.username}`);
}
```

### Like/Unlike with Optimistic UI

```typescript
// features/engagement/like-button.tsx
'use client';

import { useState, useTransition } from 'react';
import { HeartIcon } from '@heroicons/react/24/outline';
import { HeartIcon as HeartSolid } from '@heroicons/react/24/solid';
import { toggleLike } from './actions';

interface LikeButtonProps {
  postId: string;
  initialLiked: boolean;
  initialCount: number;
}

export function LikeButton({ postId, initialLiked, initialCount }: LikeButtonProps) {
  const [isPending, startTransition] = useTransition();
  const [optimisticState, setOptimisticState] = useState({
    liked: initialLiked,
    count: initialCount,
  });

  const handleClick = () => {
    // Optimistic update
    const newLiked = !optimisticState.liked;
    setOptimisticState({
      liked: newLiked,
      count: newLiked ? optimisticState.count + 1 : optimisticState.count - 1,
    });

    startTransition(async () => {
      try {
        await toggleLike(postId);
      } catch (error) {
        // Rollback on error
        setOptimisticState({
          liked: initialLiked,
          count: initialCount,
        });
      }
    });
  };

  return (
    <button
      onClick={handleClick}
      disabled={isPending}
      className="flex items-center gap-2 group disabled:opacity-50"
    >
      <div className={`p-2 rounded-full group-hover:bg-red-50 transition ${optimisticState.liked ? 'text-red-500' : 'text-gray-500'}`}>
        {optimisticState.liked ? (
          <HeartSolid className="w-6 h-6" />
        ) : (
          <HeartIcon className="w-6 h-6" />
        )}
      </div>
      <span className={`text-sm ${optimisticState.liked ? 'text-red-500' : 'text-gray-500'}`}>
        {optimisticState.count}
      </span>
    </button>
  );
}

// features/engagement/actions.ts
'use server';

import { prisma } from '@/shared/lib/db';
import { getServerSession } from 'next-auth';
import { authOptions } from '@/app/api/auth/[...nextauth]/route';
import { createNotification } from '@/lib/notifications';

export async function toggleLike(postId: string) {
  const session = await getServerSession(authOptions);
  if (!session?.user) throw new Error('Unauthorized');

  const userId = session.user.id;
  
  const existing = await prisma.like.findUnique({
    where: {
      userId_postId: { userId, postId },
    },
  });

  if (existing) {
    // Unlike
    await prisma.$transaction([
      prisma.like.delete({ where: { id: existing.id } }),
      prisma.post.update({
        where: { id: postId },
        data: { likesCount: { decrement: 1 } },
      }),
    ]);
    
    return { liked: false };
  } else {
    // Like
    await prisma.$transaction(async (tx) => {
      await tx.like.create({
        data: { userId, postId },
      });
      
      await tx.post.update({
        where: { id: postId },
        data: { likesCount: { increment: 1 } },
      });
      
      // Create notification (don't block on failure)
      const post = await tx.post.findUnique({
        where: { id: postId },
        select: { authorId: true },
      });
      
      if (post && post.authorId !== userId) {
        await createNotification({
          recipientId: post.authorId,
          actorId: userId,
          type: 'LIKE',
          entityType: 'post',
          entityId: postId,
        });
      }
    });
    
    return { liked: true };
  }
}
```

## 48.4 Real-time Interactions

Live updates for social engagement using WebSockets.

### Socket.io Room Management

```typescript
// app/api/socket/route.ts
import { Server as ServerIO } from 'socket.io';
import { NextResponse } from 'next/server';
import { prisma } from '@/shared/lib/db';

export const dynamic = 'force-dynamic';

export async function GET(req: Request) {
  if ((global as any).io) {
    return NextResponse.json({ success: true });
  }

  const io = new ServerIO({
    path: '/api/socket/io',
    cors: { origin: '*' },
  });

  io.on('connection', (socket) => {
    console.log('Client connected:', socket.id);

    // Join user-specific room for notifications
    socket.on('authenticate', async (token: string) => {
      try {
        // Verify token and get userId
        const userId = await verifySocketToken(token);
        socket.data.userId = userId;
        socket.join(`user:${userId}`);
        
        // Join rooms for posts they're viewing
        socket.on('view-post', (postId: string) => {
          socket.join(`post:${postId}`);
        });
        
        socket.on('leave-post', (postId: string) => {
          socket.leave(`post:${postId}`);
        });
      } catch (e) {
        socket.disconnect();
      }
    });

    // Handle typing indicators in comments
    socket.on('typing-comment', ({ postId, isTyping }: { postId: string; isTyping: boolean }) => {
      socket.to(`post:${postId}`).emit('user-typing', {
        userId: socket.data.userId,
        isTyping,
      });
    });
  });

  (global as any).io = io;
  return NextResponse.json({ success: true });
}

// Hook for real-time likes/comments
// hooks/use-realtime-engagement.ts
'use client';

import { useEffect } from 'react';
import { useSocket } from './use-socket';

export function useRealtimeEngagement(postId: string, onUpdate: (data: any) => void) {
  const { socket, isConnected } = useSocket();

  useEffect(() => {
    if (!isConnected || !socket) return;

    // Join post room
    socket.emit('view-post', postId);

    // Listen for engagement updates
    socket.on('new-like', (data) => {
      if (data.postId === postId) {
        onUpdate({ type: 'like', data });
      }
    });

    socket.on('new-comment', (data) => {
      if (data.postId === postId) {
        onUpdate({ type: 'comment', data });
      }
    });

    return () => {
      socket.emit('leave-post', postId);
      socket.off('new-like');
      socket.off('new-comment');
    };
  }, [socket, isConnected, postId, onUpdate]);
}
```

## 48.5 Notification System

Reliable delivery infrastructure for social alerts.

### Notification Architecture

```typescript
// lib/notifications/index.ts
import { prisma } from '@/shared/lib/db';
import { redis } from '@/shared/lib/redis';
import { Queue } from 'bullmq';

const notificationQueue = new Queue('notifications', { connection: redis });

interface CreateNotificationInput {
  recipientId: string;
  actorId: string;
  type: 'LIKE' | 'COMMENT' | 'FOLLOW' | 'MENTION' | 'REPOST';
  entityType: 'post' | 'user' | 'comment';
  entityId: string;
  metadata?: Record<string, any>;
}

export async function createNotification(input: CreateNotificationInput) {
  // Check user preferences
  const prefs = await prisma.userPreferences.findUnique({
    where: { userId: input.recipientId },
  });

  if (prefs?.[`notify_${input.type.toLowerCase()}`] === false) {
    return; // User disabled this notification type
  }

  // Create in database
  const notification = await prisma.notification.create({
    data: {
      ...input,
      read: false,
    },
    include: {
      actor: {
        select: {
          id: true,
          username: true,
          name: true,
          avatar: true,
        },
      },
    },
  });

  // Add to delivery queue
  await notificationQueue.add('deliver', {
    notificationId: notification.id,
    userId: input.recipientId,
  }, {
    attempts: 3,
    backoff: { type: 'exponential', delay: 2000 },
  });

  // Real-time push via WebSocket
  const io = (global as any).io;
  if (io) {
    io.to(`user:${input.recipientId}`).emit('notification', notification);
  }

  // Update unread count cache
  await redis.hincrby(`notifications:${input.recipientId}`, 'unread', 1);

  return notification;
}

// Worker for push/email delivery
// workers/notification-worker.ts
import { Worker } from 'bullmq';
import { sendPushNotification } from '@/lib/push';
import { sendEmail } from '@/lib/email';

const worker = new Worker('notifications', async (job) => {
  const { notificationId, userId } = job.data;
  
  const notification = await prisma.notification.findUnique({
    where: { id: notificationId },
    include: {
      recipient: { select: { email: true, pushSubscription: true } },
      actor: { select: { name: true } },
    },
  });

  if (!notification) return;

  // Send push notification if subscribed
  if (notification.recipient.pushSubscription) {
    await sendPushNotification(notification.recipient.pushSubscription, {
      title: notification.actor.name,
      body: getNotificationText(notification),
      icon: notification.actor.avatar || '/default-avatar.png',
      data: { url: getNotificationUrl(notification) },
    });
  }

  // Send email if user offline for > 5 minutes
  const lastSeen = await redis.get(`user:last_seen:${userId}`);
  if (!lastSeen || Date.now() - parseInt(lastSeen) > 5 * 60 * 1000) {
    await sendEmail({
      to: notification.recipient.email,
      subject: getNotificationSubject(notification),
      html: renderEmailTemplate(notification),
    });
  }
});
```

## 48.6 Content Moderation

AI-powered safety and community guidelines enforcement.

### Automated Content Filtering

```typescript
// lib/moderation/index.ts
import { moderateText } from './text';
import { moderateImage } from './vision';

interface ModerationResult {
  flagged: boolean;
  labels: string[];
  toxicityScore: number;
  categories: {
    hate: boolean;
    harassment: boolean;
    selfHarm: boolean;
    sexual: boolean;
    violence: boolean;
  };
}

export async function moderateContent(text?: string, imageUrls?: string[]): Promise<ModerationResult> {
  const results: Partial<ModerationResult>[] = [];

  if (text) {
    const textResult = await moderateText(text);
    results.push(textResult);
  }

  if (imageUrls && imageUrls.length > 0) {
    const imageResults = await Promise.all(
      imageUrls.map(url => moderateImage(url))
    );
    results.push(...imageResults);
  }

  // Aggregate results
  const flagged = results.some(r => r.flagged);
  const labels = results.flatMap(r => r.labels || []);
  const maxToxicity = Math.max(...results.map(r => r.toxicityScore || 0));

  return {
    flagged,
    labels: [...new Set(labels)],
    toxicityScore: maxToxicity,
    categories: {
      hate: results.some(r => r.categories?.hate),
      harassment: results.some(r => r.categories?.harassment),
      selfHarm: results.some(r => r.categories?.selfHarm),
      sexual: results.some(r => r.categories?.sexual),
      violence: results.some(r => r.categories?.violence),
    },
  };
}

// Implementation using OpenAI Moderation API
// lib/moderation/text.ts
import OpenAI from 'openai';

const openai = new OpenAI();

export async function moderateText(text: string) {
  const response = await openai.moderations.create({ input: text });
  const result = response.results[0];

  return {
    flagged: result.flagged,
    labels: result.categories
      .filter(c => c.score > 0.5)
      .map(c => c.category),
    toxicityScore: Math.max(...Object.values(result.category_scores)),
    categories: {
      hate: result.categories.hate || result.categories['hate/threatening'],
      harassment: result.categories.harassment || result.categories['harassment/threatening'],
      selfHarm: result.categories['self-harm'] || result.categories['self-harm/intent'],
      sexual: result.categories.sexual || result.categories['sexual/minors'],
      violence: result.categories.violence || result.categories['violence/graphic'],
    },
  };
}
```

### Moderation Queue & Human Review

```typescript
// app/api/webhooks/moderation/route.ts
import { NextResponse } from 'next/server';
import { prisma } from '@/shared/lib/db';

export async function POST(request: Request) {
  const payload = await request.json();
  
  // Handle human moderator decisions
  if (payload.action === 'approve') {
    await prisma.post.update({
      where: { id: payload.postId },
      data: { 
        moderationStatus: 'APPROVED',
        moderationLabels: { ...payload.labels, reviewedBy: payload.moderatorId }
      },
    });
    
    // Fan-out now that it's approved
    const post = await prisma.post.findUnique({
      where: { id: payload.postId },
      include: { author: { include: { followers: true } } },
    });
    
    if (post) {
      await fanOutPost(post);
    }
  } else if (payload.action === 'reject') {
    await prisma.post.update({
      where: { id: payload.postId },
      data: { 
        moderationStatus: 'REJECTED',
        rejectionReason: payload.reason,
      },
    });
    
    // Notify user
    await createNotification({
      recipientId: payload.authorId,
      type: 'CONTENT_REJECTED',
      entityType: 'post',
      entityId: payload.postId,
      metadata: { reason: payload.reason },
    });
  }

  return NextResponse.json({ success: true });
}
```

## 48.7 Scaling Strategies

Handling viral content and high-write workloads.

### Database Optimization

```typescript
// lib/db/optimization.ts
// Connection pooling for high concurrency
import { Pool } from '@neondatabase/serverless';
import { PrismaNeon } from '@prisma/adapter-neon';

const pool = new Pool({ 
  connectionString: process.env.DATABASE_URL,
  maxConnections: 20, // Limit for serverless
  connectionTimeoutMillis: 5000,
});

// Read replicas for feed generation
const readPool = process.env.DATABASE_READ_URL 
  ? new Pool({ 
      connectionString: process.env.DATABASE_READ_URL,
      maxConnections: 30,
    })
  : pool;

// Caching hot paths
import { unstable_cache } from 'next/cache';

export const getCachedUserStats = unstable_cache(
  async (userId: string) => {
    return prisma.user.findUnique({
      where: { id: userId },
      select: {
        _count: {
          select: {
            posts: true,
            followers: { where: { status: 'ACCEPTED' } },
            following: { where: { status: 'ACCEPTED' } },
          },
        },
      },
    });
  },
  ['user-stats'],
  { revalidate: 60, tags: ['user-stats'] }
);

// Batch operations for high-volume writes
export async function batchCreateFeedItems(items: any[]) {
  // Use COPY command for bulk inserts if using raw SQL
  // Or chunk Prisma createMany
  const chunks = chunkArray(items, 1000);
  
  for (const chunk of chunks) {
    await prisma.feedItem.createMany({
      data: chunk,
      skipDuplicates: true,
    });
  }
}

function chunkArray<T>(array: T[], size: number): T[][] {
  return Array.from({ length: Math.ceil(array.length / size) }, (_, i) =>
    array.slice(i * size, i * size + size)
  );
}
```

### Rate Limiting & Abuse Prevention

```typescript
// middleware/rate-limit.ts
import { Ratelimit } from '@upstash/ratelimit';
import { Redis } from '@upstash/redis';

const ratelimit = {
  post: new Ratelimit({
    redis: Redis.fromEnv(),
    limiter: Ratelimit.slidingWindow(5, '1 m'), // 5 posts per minute
  }),
  follow: new Ratelimit({
    redis: Redis.fromEnv(),
    limiter: Ratelimit.slidingWindow(20, '1 h'), // 20 follows per hour
  }),
  like: new Ratelimit({
    redis: Redis.fromEnv(),
    limiter: Ratelimit.slidingWindow(100, '1 m'), // 100 likes per minute
  }),
};

export async function checkRateLimit(
  userId: string, 
  action: keyof typeof ratelimit
) {
  const identifier = `${userId}:${action}`;
  const { success, limit, reset, remaining } = await ratelimit[action].limit(identifier);
  
  if (!success) {
    throw new Error(`Rate limit exceeded. Try again in ${Math.ceil((reset - Date.now()) / 1000)}s`);
  }
  
  return { remaining, reset };
}
```

## Key Takeaways from Chapter 48

1. **Social Graph Modeling**: Use adjacency list pattern in PostgreSQL with compound indexes on `(followerId, followingId)` for efficient follow/unfollow checks. Store denormalized counts (followers, following) on the User table to avoid expensive `COUNT(*)` queries on profile pages, updating them asynchronously via database triggers or application logic.

2. **Feed Architecture**: Implement hybrid fan-out strategy: pre-compute feeds via `FeedItem` table for users with <10k followers (write-time fan-out), use pull-based generation for celebrities (>10k followers) querying recent posts from followed accounts. Use cursor-based pagination with `(userId, createdAt)` indexes rather than offset-based pagination for infinite scroll performance.

3. **Content Moderation Pipeline**: Integrate AI moderation APIs (OpenAI, AWS Rekognition) at post creation time, storing results in `moderationStatus` and `moderationLabels` columns. Queue posts flagged as questionable for human review, only fanning out approved content to feeds. Implement soft deletion preserving relationships while removing content.

4. **Real-time Engagement**: Use Socket.io rooms named by post ID (`post:${postId}`) for live comment/like updates, and user-specific rooms (`user:${userId}`) for notifications. Implement optimistic UI for like buttons with immediate state updates and rollback on server error, using `useTransition` for smooth UX during mutations.

5. **Notification Delivery**: Create notification records in PostgreSQL for persistence, enqueue to BullMQ for reliable delivery to push/email services. Cache unread counts in Redis using `HINCRBY` for fast badge updates, synchronizing with database counts periodically. Check user preferences before creating notifications to respect mute settings.

6. **High-Write Optimization**: Use connection pooling with 20-max connections for serverless environments, implement read replicas for feed generation queries. Batch insert feed items in chunks of 1000 to prevent database timeouts during fan-out. Use `createMany` with `skipDuplicates` for idempotent feed population.

7. **Abuse Prevention**: Apply tiered rate limiting per action type—strict limits on follows (20/hour) to prevent spam, moderate on posts (5/minute), generous on likes (100/minute). Use Redis-based sliding window counters with Upstash Ratelimit, checking limits before expensive database operations in Server Actions.

## Coming Up Next

**Chapter 49: Edge Computing**

Having explored full-stack application architectures, Chapter 49 dives into edge-native patterns. You'll learn to deploy compute at the edge using Vercel Edge Functions and Middleware, implement geographically distributed data caching, handle authentication at the edge for zero-latency verification, and architect applications that run closest to users globally. Understand the tradeoffs between Node.js and Edge runtimes, and when to choose each for optimal performance.